In [99]:
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [122]:
# load data 
df = pd.read_parquet('processed_data.parquet')

In [ ]:
pedpedal_action_ind

In [118]:
# filter with prefix pedpedal_action_ind
p = df.filter(like='pedpedal_action_ind')
p.columns.tolist()

['pedpedal_action_ind_0',
 'pedpedal_action_ind_AGAINST TRAFFIC',
 'pedpedal_action_ind_CROSSING - AGAINST SIGNAL',
 'pedpedal_action_ind_CROSSING - CONTROLS PRESENT (NOT AT INTERSECTION)',
 'pedpedal_action_ind_CROSSING - NO CONTROLS (AT INTERSECTION)',
 'pedpedal_action_ind_CROSSING - NO CONTROLS (NOT AT INTERSECTION)',
 'pedpedal_action_ind_CROSSING - WITH SIGNAL',
 'pedpedal_action_ind_ENTER FROM DRIVE/ALLEY',
 'pedpedal_action_ind_INTOXICATED PED/PEDAL',
 'pedpedal_action_ind_NO ACTION',
 'pedpedal_action_ind_NOT AT INTERSECTION',
 'pedpedal_action_ind_OTHER ACTION',
 'pedpedal_action_ind_PARKED VEHICLE',
 'pedpedal_action_ind_PLAYING IN ROADWAY',
 'pedpedal_action_ind_PLAYING/WORKING ON VEHICLE',
 'pedpedal_action_ind_SCHOOL BUS (WITHIN 50 FT.)',
 'pedpedal_action_ind_STANDING IN ROADWAY',
 'pedpedal_action_ind_TO/FROM DISABLED VEHICLE',
 'pedpedal_action_ind_TURNING LEFT',
 'pedpedal_action_ind_TURNING RIGHT',
 'pedpedal_action_ind_UNKNOWN/NA',
 'pedpedal_action_ind_WAITING FOR 

In [124]:
input_cols = ['district_y', 'crash_hour_ind', 'posted_speed_limit_ind', 'lighting_condition_ind_DAYLIGHT','crash_day_of_week_ind', 
              'target', 'pedpedal_action_ind_CROSSING - AGAINST SIGNAL', 'pedpedal_action_ind_CROSSING - CONTROLS PRESENT (NOT AT INTERSECTION)', 
              'pedpedal_action_ind_CROSSING - NO CONTROLS (AT INTERSECTION)', 'pedpedal_action_ind_CROSSING - NO CONTROLS (NOT AT INTERSECTION)',
              'pedpedal_action_ind_CROSSING - WITH SIGNAL']



df = df[input_cols]

In [113]:
df.head(2)

,district_y,crash_hour_ind,posted_speed_limit_ind,lighting_condition_ind_DAYLIGHT,crash_day_of_week_ind,target
0,0.0,18,30,0,3,1
1,20.0,17,30,0,3,1


In [125]:
# Split the data into X and y
X = df.drop(columns=['target']).to_numpy() 
y = df['target'].to_numpy() 

In [126]:
df['target'].value_counts()

target
1    11937
0     7411
2     3265
Name: count, dtype: int64

Naive Bayes can be extended to handle multi-class classification problems. In this context, it calculates the probability of each class given the features and selects the class with the highest probability as the predicted class.

Logistic Regression can be extended to handle multi-class classification tasks. One common approach is the one-vs-rest (OvR) strategy, where separate binary classifiers are trained for each class. Each classifier is trained to distinguish between one class and the rest. Alternatively, the one-vs-one (OvO) strategy trains a binary classifier for each pair of classes.

SVM: SVM can be adapted to handle multi-class classification using either the one-vs-one (OvO) or one-vs-rest (OvR) strategy. In OvO, a binary classifier is trained for each pair of classes, and the class with the most votes is chosen. In OvR, separate binary classifiers are trained for each class, where each classifier distinguishes between one class and the rest.

Random Forest: Ensemble learning method that combines the strengths of decision trees with randomization to achieve high predictive accuracy and generalization performance.

Gradient Boosting: Builds an ensemble of weak learners, optimizing them using gradient descent to minimize a loss function and achieve strong predictive performance.


In [127]:
from sklearn.naive_bayes import GaussianNB

def naive_bayes_accuracy_per_class(X, y, n_splits=5, n_repeats=2):
    nb_classifier = GaussianNB()

    # Perform cross-validation and get predicted labels for each sample
    y_pred = cross_val_predict(nb_classifier, X, y, cv=n_splits, n_jobs=-1)

    # accuracy for each class
    accuracy_per_class = []
    for class_label in np.unique(y):
        class_indices = np.where(y == class_label)[0]
        class_accuracy = accuracy_score(y[class_indices], y_pred[class_indices])
        accuracy_per_class.append(class_accuracy)

    return accuracy_per_class

# Example:
accuracies = naive_bayes_accuracy_per_class(X, y)

naive_bayes_results = pd.DataFrame({'Class': range(len(accuracies)), 'Accuracy': accuracies})


In [128]:
from sklearn.linear_model import LogisticRegression

def logistic_regression_accuracy_per_class(X, y, n_splits=5, n_repeats=2):
    lr_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')

    # cross-validation and get predicted labels for each sample
    y_pred = cross_val_predict(lr_classifier, X, y, cv=n_splits, n_jobs=-1)

    # accuracy for each class
    accuracy_per_class = []
    for class_label in np.unique(y):
        class_indices = np.where(y == class_label)[0]
        class_accuracy = accuracy_score(y[class_indices], y_pred[class_indices])
        accuracy_per_class.append(class_accuracy)

    return accuracy_per_class

# Example:
accuracies = logistic_regression_accuracy_per_class(X, y)

logistic_results = pd.DataFrame({'Class': range(len(accuracies)), 'Accuracy': accuracies})


/opt/anaconda3/envs/chicago/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/chicago/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [129]:
from sklearn.svm import SVC

def svm_accuracy_per_class(X, y, n_splits=5, n_repeats=2):
    svm_classifier = SVC()

    # cross-validation and get predicted labels for each sample
    y_pred = cross_val_predict(svm_classifier, X, y, cv=n_splits, n_jobs=-1)

    # accuracy for each class
    accuracy_per_class = []
    for class_label in np.unique(y):
        class_indices = np.where(y == class_label)[0]
        class_accuracy = accuracy_score(y[class_indices], y_pred[class_indices])
        accuracy_per_class.append(class_accuracy)

    return accuracy_per_class

# Example:
accuracies = svm_accuracy_per_class(X, y)

svm_results = pd.DataFrame({'Class': range(len(accuracies)), 'Accuracy': accuracies})


In [130]:
from sklearn.ensemble import RandomForestClassifier

def random_forest_accuracy_per_class(X, y, n_splits=5, n_repeats=2):
    rf_classifier = RandomForestClassifier()

    # cross-validation and get predicted labels for each sample
    y_pred = cross_val_predict(rf_classifier, X, y, cv=n_splits, n_jobs=-1)

    # accuracy for each class
    accuracy_per_class = []
    for class_label in np.unique(y):
        class_indices = np.where(y == class_label)[0]
        class_accuracy = accuracy_score(y[class_indices], y_pred[class_indices])
        accuracy_per_class.append(class_accuracy)

    return accuracy_per_class

# Example:
accuracies = random_forest_accuracy_per_class(X, y)

random_forest_results = pd.DataFrame({'Class': range(len(accuracies)), 'Accuracy': accuracies})


In [131]:
from sklearn.ensemble import GradientBoostingClassifier

def gradient_boosting_accuracy_per_class(X, y, n_splits=5, n_repeats=2):
    gb_classifier = GradientBoostingClassifier()

    # cross-validation and predicted class for each sample
    y_pred = cross_val_predict(gb_classifier, X, y, cv=n_splits, n_jobs=-1)

    # accuracy for each class
    accuracy_per_class = []
    for class_label in np.unique(y):
        class_indices = np.where(y == class_label)[0]
        class_accuracy = accuracy_score(y[class_indices], y_pred[class_indices])
        accuracy_per_class.append(class_accuracy)

    return accuracy_per_class

# Example:
accuracies = gradient_boosting_accuracy_per_class(X, y)

gradient_bosting_results = pd.DataFrame({'Class': range(len(accuracies)), 'Accuracy': accuracies})



In [132]:
results = pd.concat([naive_bayes_results, logistic_results, svm_results, random_forest_results, gradient_bosting_results], axis=1)

In [133]:
results

,Class,Accuracy,Class,Accuracy,Class,Accuracy,Class,Accuracy,Class,Accuracy
0,0,0.182162,0,0.001079,0,0.0,0,0.297126,0,0.020105
1,1,0.775907,1,0.999246,1,1.0,1,0.645137,1,0.984083
2,2,0.091884,2,0.000000,2,0.0,2,0.085145,2,0.005513
